# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output2'
TRAIT = "Lactose-Intolerance"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

utils.py has been loaded


# 2. Data preprocessing and selection

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [2]:
dataset = 'GEO'
trait_subdir = "Lactose-Intolerance"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE136395', 'GSE138297']

In [11]:
cohort = accession_num = "GSE138297"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Lactose-Intolerance\\GSE138297\\GSE138297_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Lactose-Intolerance\\GSE138297\\GSE138297_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [12]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"The host response of IBS patients to allogenic and autologous faecal microbiota transfer"
!Series_summary	"In this randomised placebo-controlled trial, irritable bowel syndrome (IBS) patients were treated with faecal material from a healthy donor (n=8, allogenic FMT) or with their own faecal microbiota (n=8, autologous FMT). The faecal transplant was administered by whole colonoscopy into the caecum (30 g of stool in 150 ml sterile saline). Two weeks before the FMT (baseline) as well as two and eight weeks after the FMT, the participants underwent a sigmoidoscopy, and biopsies were collected at a standardised location (20-25 cm from the anal verge at the crossing with the arteria iliaca communis) from an uncleansed sigmoid. In patients treated with allogenic FMT, predominantly immune response-related genes sets were induced, with the strongest response two weeks after FMT. In patients treated with autologous FMT, predominantly metabolism-related gene sets were affected."

In [13]:
clinical_data

,!Sample_geo_accession,GSM4104672,GSM4104673,GSM4104674,GSM4104675,GSM4104676,GSM4104677,GSM4104678,GSM4104679,GSM4104680,...,GSM4104707,GSM4104708,GSM4104709,GSM4104710,GSM4104711,GSM4104712,GSM4104713,GSM4104714,GSM4104715,GSM4104716
0,!Sample_characteristics_ch1,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,...,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon,tissue: uncleansed colon
1,!Sample_characteristics_ch1,"sex (female=1, male=0): 1","sex (female=1, male=0): 1","sex (female=1, male=0): 1","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0",...,"sex (female=1, male=0): 1","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 0","sex (female=1, male=0): 1","sex (female=1, male=0): 1","sex (female=1, male=0): 1"
2,!Sample_characteristics_ch1,subjectid: 1,subjectid: 1,subjectid: 1,subjectid: 2,subjectid: 2,subjectid: 2,subjectid: 3,subjectid: 3,subjectid: 3,...,subjectid: 13,subjectid: 14,subjectid: 14,subjectid: 14,subjectid: 15,subjectid: 15,subjectid: 15,subjectid: 16,subjectid: 16,subjectid: 16
3,!Sample_characteristics_ch1,age (yrs): 49,age (yrs): 49,age (yrs): 49,age (yrs): 21,age (yrs): 21,age (yrs): 21,age (yrs): 31,age (yrs): 31,age (yrs): 31,...,age (yrs): 23,age (yrs): 50,age (yrs): 50,age (yrs): 50,age (yrs): 32,age (yrs): 32,age (yrs): 32,age (yrs): 38,age (yrs): 38,age (yrs): 38
4,!Sample_characteristics_ch1,moment of sampling (pre/post intervention): Ba...,moment of sampling (pre/post intervention): 2 ...,moment of sampling (pre/post intervention): 8 ...,moment of sampling (pre/post intervention): Ba...,moment of sampling (pre/post intervention): 2 ...,moment of sampling (pre/post intervention): 8 ...,moment of sampling (pre/post intervention): Ba...,moment of sampling (pre/post intervention): 2 ...,moment of sampling (pre/post intervention): 8 ...,...,moment of sampling (pre/post intervention): 8 ...,moment of sampling (pre/post intervention): Ba...,moment of sampling (pre/post intervention): 2 ...,moment of sampling (pre/post intervention): 8 ...,moment of sampling (pre/post intervention): Ba...,moment of sampling (pre/post intervention): 2 ...,moment of sampling (pre/post intervention): 8 ...,moment of sampling (pre/post intervention): Ba...,moment of sampling (pre/post intervention): 2 ...,moment of sampling (pre/post intervention): 8 ...
5,!Sample_characteristics_ch1,"time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...",...,"time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas...","time of sampling: Morning, after overnight fas..."
6,!Sample_characteristics_ch1,experimental condition: Allogenic FMT,experimental condition: 

In [14]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: uncleansed colon'],
 1: ['sex (female=1, male=0): 1', 'sex (female=1, male=0): 0'],
 2: ['subjectid: 1',
  'subjectid: 2',
  'subjectid: 3',
  'subjectid: 4',
  'subjectid: 5',
  'subjectid: 6',
  'subjectid: 7',
  'subjectid: 8',
  'subjectid: 10',
  'subjectid: 11',
  'subjectid: 12',
  'subjectid: 13',
  'subjectid: 14',
  'subjectid: 15',
  'subjectid: 16'],
 3: ['age (yrs): 49',
  'age (yrs): 21',
  'age (yrs): 31',
  'age (yrs): 59',
  'age (yrs): 40',
  'age (yrs): 33',
  'age (yrs): 28',
  'age (yrs): 36',
  'age (yrs): 50',
  'age (yrs): 27',
  'age (yrs): 23',
  'age (yrs): 32',
  'age (yrs): 38'],
 4: ['moment of sampling (pre/post intervention): Baseline (app. 2w before intervention)',
  'moment of sampling (pre/post intervention): 2 weeks after intervention',
  'moment of sampling (pre/post intervention): 8 weeks after intervention'],
 5: ['time of sampling: Morning, after overnight fasting'],
 6: ['experimental condition: Allogenic FMT',
  'experimental cond

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [15]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Lactose-Intolerance\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Lactose-Intolerance\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [16]:
age_row = gender_row = None
convert_age = convert_gender = None

In [17]:
is_gene_availabe = True
trait_row = None
age_row = 3
gender_row = 1

trait_type = 'binary'

In [18]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [128]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue: uncleansed colon':
        return 1  
    else:
        return 0 

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age (yrs): '):
        try:
            return int(age_string.split('age (yrs): ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'sex (female=1, male=0): 1':
        return 0
    elif gender_string == 'sex (female=1, male=0): 0':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: uncleansed colon'
example_age = 'age (yrs): 49'
example_gender = 'sex (female=1, male=0): 0'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, 49, 1)

In [129]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM4104672,GSM4104673,GSM4104674,GSM4104675,GSM4104676,GSM4104677,GSM4104678,GSM4104679,GSM4104680,GSM4104681,...,GSM4104707,GSM4104708,GSM4104709,GSM4104710,GSM4104711,GSM4104712,GSM4104713,GSM4104714,GSM4104715,GSM4104716
Lactose-Intolerance,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Age,49,49,49,21,21,21,31,31,31,59,...,23,50,50,50,32,32,32,38,38,38
Gender,0,0,0,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,0,0,0


### Genetic data preprocessing and final filtering

In [130]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4104672,GSM4104673,GSM4104674,GSM4104675,GSM4104676,GSM4104677,GSM4104678,GSM4104679,GSM4104680,GSM4104681,...,GSM4104707,GSM4104708,GSM4104709,GSM4104710,GSM4104711,GSM4104712,GSM4104713,GSM4104714,GSM4104715,GSM4104716
ID,,,,,,,,,,,,,,,,,,,,,
16650001,1.605655,1.843828,1.899724,1.617481,1.920638,3.633856,1.934084,1.573835,1.409345,2.211837,...,3.874015,2.754243,0.907163,1.654553,0.595274,1.030531,1.829221,2.945017,3.135033,3.589383
16650003,2.279692,2.658503,1.714000,3.420702,2.070772,1.712379,1.600772,2.391815,1.506170,1.935247,...,1.538323,2.271402,2.014419,1.721832,1.707934,2.081874,2.134537,1.668077,1.741112,1.988065
16650005,1.574519,1.461831,1.534462,1.327591,2.968294,1.456212,1.499459,2.294922,2.467470,2.137064,...,2.205266,1.238521,1.626757,1.765212,2.255707,1.892722,1.583878,1.790000,1.904948,2.471706
16650007,1.630632,1.678148,1.366905,1.685712,1.274810,2.059780,2.571425,1.768870,2.069762,1.733600,...,1.526955,3.184611,2.405612,1.292599,3.129706,1.468203,1.940357,1.344478,2.081564,2.053391
16650009,0.922291,1.683431,1.366627,1.115257,1.310950,1.362507,1.347023,1.358300,1.193375,1.046035,...,1.072729,1.293214,1.218343,1.324958,3.572712,1.295168,1.050756,1.094166,0.927726,0.924413


In [131]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['16650001',
 '16650003',
 '16650005',
 '16650007',
 '16650009',
 '16650011',
 '16650013',
 '16650015',
 '16650017',
 '16650019',
 '16650021',
 '16650023',
 '16650025',
 '16650027',
 '16650029',
 '16650031',
 '16650033',
 '16650035',
 '16650037',
 '16650041']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [132]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['16650001', '16650003', '16650005', '16650007', '16650009', '16650011', '16650013', '16650015', '16650017', '16650019', '16650021', '16650023', '16650025', '16650027', '16650029', '16650031', '16650033', '16650035', '16650037', '16650041']\n"


If not required, jump directly to the gene normalization step

In [133]:
requires_gene_mapping = False

In [134]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [135]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [136]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [137]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1000 input query terms found no hit:	['16650001', '16650003', '16650005', '16650007', '16650009', '16650011', '16650013', '16650015', '16
1000 input query terms found no hit:	['16652093', '16652095', '16652097', '16652099', '16652101', '16652103', '16652105', '16652107', '16
1000 input query terms found no hit:	['16654185', '16654187', '16654189', '16654191', '16654193', '16654195', '16654197', '16654199', '16
1000 input query terms found no hit:	['16656247', '16656249', '16656251', '16656253', '16656255', '16656257', '16656259', '16656261', '16
1000 input query terms found no hit:	['16661723', '16661726', '16661730', '16661770', '16661773', '16661776', '16661778', '16661782', '16
1000 input query terms found no hit:	['16672006', '16672027', '16672043', '16672052', '16672072', '16672075', '16672080', '16672082', '16
1000 input query terms found no hit:	['16682148', '16682161', '16682175', '16682203', '16682219', '16682225', '16682259', '16682275', '16
1000 input query terms found no hi

In [138]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [139]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 45 samples.


In [140]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lactose-Intolerance', the least common label is '1.0' with 45 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Lactose-Intolerance' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 28.0
  50% (Median): 36.0
  75%: 49.0
Min: 21.0
Max: 59.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 21 occurrences. This represents 46.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [141]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [142]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [143]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [144]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [145]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
